In [1]:
import easyocr
import os
import torch
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric
import re
import cv2 as cv
import onnxruntime
from torchvision import transforms

In [2]:
# !pip install onnx
# !pip install onnxruntime-gpu

In [3]:
torch.cuda.is_available()

True

In [4]:
model = 'crnn_30k_lr_25e-2' #folder name of model
iteration = 'crnn_30k_lr_25e-2' # model .pth file name

model_path = os.path.join('trainer', 'saved_models',model)
reader = easyocr.Reader(lang_list = ['en','th'],
                    model_storage_directory = model_path,
                    user_network_directory = os.path.join('my_model','user_network'),
                    recog_network  = iteration,
                    config_path = 'crnn', #configuration file name use to get character_list only  (so able to use for every model)
                    gpu = True)
# reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )

--------------------
model_path: trainer\saved_models\crnn_30k_lr_25e-2\crnn_30k_lr_25e-2.pth
config: my_model\user_network\crnn.yaml
--------------------
Transformation: None
FeatureExtraction: VGG
SequenceModeling: BiLSTM
Prediction: CTC


In [13]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
random_file_path = filepaths[0]

img = plt.imread(random_file_path)
# print (f'image befor reader: {img[0][0]}')
result = reader.recognize(random_file_path)
result

[([[0, 0], [103, 0], [103, 59], [0, 59]], 'วอห์น', 0.5715873732040726)]

In [14]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()if tensor.requires_grad else tensor.cpu().numpy()

def custom_mean(x):
    return x.prod()**(2.0/np.sqrt(len(x)))

import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from easyocr.utils import reformat_input
from easyocr.utils import get_image_list
from easyocr.recognition import AlignCollate
from easyocr.recognition import ListDataset
import torch.nn.functional as F
from easyocr.utils import CTCLabelConverter

filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))[:1000]
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
label_path = 'trainer/all_data/testing/testing/label.txt'
with open(label_path, encoding='utf8') as file:
    label = np.loadtxt(file,dtype=str)
start = time.time()
filepaths = filepaths[:10]
pred_list = []
gt = [] # ground truth
name_img = []

for i_img in tqdm(range(len(filepaths))):
    img, img_cv_grey = reformat_input(random_file_path)
    # print(f'reformat {img_cv_grey}')
    y_max, x_max = img_cv_grey.shape
    horizontal_list = [[0, x_max, 0, y_max]]
    for bbox in horizontal_list:
                    h_list = [bbox]
                    f_list = []
                    image_list, max_width = get_image_list(h_list, f_list, img_cv_grey, model_height = 64)
    # print(f'image_list: {image_list}')
    coord = [item[0] for item in image_list]
    img_list = [item[1] for item in image_list]
    # print(f'image_list: {img_list}')
    AlignCollate_normal = AlignCollate(imgH=64, imgW=600, keep_ratio_with_pad=True)
    test_data = ListDataset(img_list)
    # print(f'test_data {test_data}')
    test_loader = torch.utils.data.DataLoader(
            test_data, batch_size=1, shuffle=False,
            num_workers=int(1), collate_fn=AlignCollate_normal, pin_memory=True)
    # print(f'test_loader {test_loader}')
    ort_session = onnxruntime.InferenceSession("recognitionModel.onnx", providers=['CUDAExecutionProvider', 'TensorrtExecutionProvider'])
    ort_session.set_providers(['CUDAExecutionProvider', 'TensorrtExecutionProvider'])
    for image_tensors in test_loader:
        image = image_tensors.to('cuda')
        # print(f'image_tensors {to_numpy(image)}')
        start = time.time()
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image)}

        ort_outs = ort_session.run(None, ort_inputs)
        preds = torch.from_numpy(ort_outs[0])
        device = torch.device("cuda")
        batch_size = 1
        ignore_idx = []
        preds_size = torch.IntTensor([preds.size(1)] * batch_size)
        preds_prob = F.softmax(preds, dim=2)
        preds_prob = preds_prob.cpu().detach().numpy()
        preds_prob[:,:,ignore_idx] = 0.
        pred_norm = preds_prob.sum(axis=2)
        preds_prob = preds_prob/np.expand_dims(pred_norm, axis=-1)
        preds_prob = torch.from_numpy(preds_prob).float().to(device)
        result = []
        decoder = 'greedy'
        character = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\]^_`|~กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤลฦวศษสหฬอฮฯะัาำิีึืุูเแโใไๅๆ็่้๊๋์ํ๐๑๒๓๔๕๖๗๘๙–‘ ’'
        separator_list = {}
        dict_list = {'en': 'd:\\Intern\\EasyOCR\\easyocr\\dict\\en.txt', 'th': 'd:\\Intern\\EasyOCR\\easyocr\\dict\\th.txt'}
        converter = CTCLabelConverter(character, separator_list, dict_list)
        if decoder == 'greedy':
            # Select max probabilty (greedy decoding) then decode index to character
            _, preds_index = preds_prob.max(2)
            preds_index = preds_index.view(-1)
            preds_str = converter.decode_greedy(preds_index.data.cpu().detach().numpy(), preds_size.data)
        elif decoder == 'beamsearch':
            k = preds_prob.cpu().detach().numpy()
            preds_str = converter.decode_beamsearch(k, beamWidth=5)
        elif decoder == 'wordbeamsearch':
            k = preds_prob.cpu().detach().numpy()
            preds_str = converter.decode_wordbeamsearch(k, beamWidth=5)

        preds_prob = preds_prob.cpu().detach().numpy()
        values = preds_prob.max(axis=2)
        indices = preds_prob.argmax(axis=2)
        preds_max_prob = []
        for v,i in zip(values, indices):
            max_probs = v[i!=0]
            if len(max_probs)>0:
                preds_max_prob.append(max_probs)
            else:
                preds_max_prob.append(np.array([0]))

        for pred, pred_max_prob in zip(preds_str, preds_max_prob):
            confidence_score = custom_mean(pred_max_prob)
            result.append([pred, confidence_score])
        try:
            pred_list.append(result)
        except:
            pred_list.append('')

    label_img = label[i][1]
    name_img.append(label[i][0])
    gt.append(label_img)

end = time.time()
print(f'time: {end - start}')


  0%|          | 0/10 [00:03<?, ?it/s]


AttributeError: 'str' object has no attribute 'append'

In [ ]:
tmp = name_img.copy()
output, false_list = eval_ocr_metric(pred_list, gt, tmp)
# output, false_list = eval_ocr_metric(pred, gt, name_img)
for k,v in output.items():
    print(f'{k}: {v}')

In [7]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# # filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))[:1000]
# filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)
# start = time.time()
# # filepaths = filepaths[:10]
# pred = []
# gt = [] # ground truth
# name_img = []
# for i in tqdm(range(len(filepaths))):
#     label_img = label[i][1]
#     name_img.append(label[i][0])
#     result = reader.recognize(filepaths[i],detail=0)
#     try:
#         pred.append(result)
#     except:
#         pred.append('')
#     gt.append(label_img)
# end = time.time()
# print(f'time: {end - start}')

In [8]:
# tmp = name_img.copy()
# output, false_list = eval_ocr_metric(pred, gt, tmp)
# # output, false_list = eval_ocr_metric(pred, gt, name_img)
# for k,v in output.items():
#     print(f'{k}: {v}')